<a href="https://colab.research.google.com/github/JumperWacky/cycleGAN/blob/main/mnist_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pandas,numpy,random
import matplotlib.pyplot as plt

In [4]:
# Dataset类，用于data预处理，data传入等
class MnistDataset(Dataset):
  def __init__(self, csv_file):
    count = 0
    with open(csv_file) as rf:
      for line in rf:
        count += 1
        items = line.strip().split(',')
        if count == 577:
          print(line)
          print(len(items))
    self.data_df = pandas.read_csv(csv_file, header=None)
  
  def __len__(self):
    return len(self.data_df)
  
  def __getitem__(self, index):
    # 图像label one-hot编码
    label = self.data_df.iloc[index, 0]
    target = torch.zeros(10)
    target[label] = 1.0

    # 图像数据
    # iloc生成一个二维区域，values生成一个numpy.array
    image_values = torch.FloatTensor(self.data_df.iloc[index, 1:].values) / 255.0

    # 返回标签、图像数据张量及目标张量
    return label, image_values, target
  
  def plot_image(self, index):
    arr = self.data_df.iloc[index, 1:].values.reshape(28, 28)
    plt.title("label = " + str(self.data_df.iloc[index,0]))
    plt.imshow(arr, interpolation='none', cmap='Blues')
    pass

In [5]:
# load Data
mnist_dataset = MnistDataset("/content/sample_data/mnist_train.csv")
mnist_dataset.plot_image([20])

4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66,189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,226,232,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,245,129,0,0,0,0,0,24,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,191,254,97,0,0,0,0,0,114,213,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,254,254,11,0,0,0,0,34,218,243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,254,254,11,0,0,0,33,162,254,243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72,254,254,17,18,126,152,247,254,252,254,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65,239,254,254,254,254,249,194,113,189,248,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,199,213,213,193,73,0,0,232,243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,232,243,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,232,206,0,0,0,0,0

ParserError: ignored

In [6]:
def generate_random(size=4):
  random_data = torch.rand(size)
  return random_data

In [7]:
# 构造鉴别器
class Discriminator(nn.Module):
  def __init__(self) -> None:
    super().__init__()

    # 定义神经网路层
    self.model = nn.Sequential(
        nn.Linear(784, 200),
        nn.Sigmoid(),
        nn.Linear(200, 1),
        nn.Sigmoid()
    )

    # 创建损失函数
    self.loss_function = nn.MSELoss()

    # 创建优化器
    self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)

    # 计数器和进程记录
    self.counter = 0
    self.progress = []

    pass
  
  def forward(self, inputs):
    return self.model(inputs)
  
  def train(self, inputs, targets):
    # 计算网络的输出值
    outputs = self.forward(inputs)
    # 计算损失值
    loss = self.loss_function(outputs, targets)

    # 梯度归零，反向传播，更新权重
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    # 每隔10个训练样本增加一次计数器的值，并将损失值添加进列表的末尾
    self.counter += 1
    if(self.counter % 10 == 0):
      self.progress.append(loss.item())
      pass
    if(self.counter % 10000 == 0):
      print("dicriminator counter = ", self.counter)
      pass
  
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.',
        grid=True, yticks=(0,0.25,0.5))
    pass

In [ ]:
%%time
# 测试鉴别器
D = Discriminator()
for label, image_data_tensor, target_tensor in mnist_dataset:
  # 真实数据
  D.train(image_data_tensor, torch.FloatTensor([1.0]))
  # 生成数据
  D.train(generate_random(784), torch.FloatTensor([0.0]))

In [ ]:
D.plot_progress()
for i in range(4):
  image_data_tensor = mnist_dataset[random.randint(0, 60000)][1]
  print( D.forward(image_data_tensor).item() )
  print( D.forward(generate_random(784)).item() )

In [ ]:
# 构造生成器
class Generator(nn.Module):
  def __init__(self):
    super().__init__()

    # 定义神经网路层
    self.model = nn.Sequential(
        nn.Linear(1, 3),
        nn.Sigmoid(),
        nn.Linear(3, 4),
        nn.Sigmoid()
    )

    # 创建优化器
    self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)  # 使用的是生成器的parameters

    # 计数器和进程记录
    self.counter = 0
    self.progress = []

    pass
  
  def forward(self, inputs):
    return self.model(inputs)
  
  def train(self, D, inputs, targets):
    # 计算生成器网络输出
    g_output = self.forward(inputs)

    # 输入鉴别器
    d_output = D.forward(g_output)

    # 计算损失值
    loss = D.loss_function(d_output, targets)
    # 每训练10次增加计数器
    self.counter += 1
    if (self.counter % 10 == 0):
      self.progress.append(loss.item())
      pass
    if (self.counter % 10000 == 0):
      print("generator counter = ", self.counter)
      pass
    
    # 梯度归零，反向传播，更新权重
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()    # 只更新生成器的parameter

    pass
  
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.',
        grid=True, yticks=(0,0.25,0.5))
    pass
